In [6]:
!apt-get update
!apt-get install -y openjdk-21-jdk
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/java-21-openjdk-amd64/bin/java 1
!update-alternatives --config java
!java -version


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,135 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cli.github.com/packages stable/main amd64 Packages [343 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InReleas

In [2]:
# 0) Basic setup
!pip -q install pymupdf pdfplumber pytesseract sentence-transformers faiss-cpu pyserini pillow transformers accelerate bitsandbytes
import os, json, math, re, io
import fitz  # PyMuPDF
import pdfplumber
from PIL import Image
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Choose small, free models
TEXT_EMB = SentenceTransformer("BAAI/bge-small-en-v1.5")
CAPTION_EMB = SentenceTransformer("intfloat/e5-small")
# For generation, we’ll select later based on GPU


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.8/178.8 MB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 113.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 128.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.5/367.5 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.4 MB/s eta 0

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/362 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [7]:
# 1) Parse PDF into text sections and figure crops with captions
def parse_pdf(pdf_path, dpi=200):
    doc = fitz.open(pdf_path)
    pages = []
    for i in range(len(doc)):
        page = doc[i]
        text = page.get_text("blocks")  # preserves layout blocks
        pages.append({"index": i, "blocks": text})
    return doc, pages

def extract_figures_with_captions(pdf_path):
    figures = []
    with pdfplumber.open(pdf_path) as pdf:
        for pi, page in enumerate(pdf.pages):
            # Heuristic: captions often start with "Figure" or "Fig."
            text = page.extract_text() or ""
            # Simple heuristic: find lines starting with Fig/Figure
            caption_candidates = [l for l in (text.split("\n")) if re.match(r"^\s*(Fig\.|Figure)\s*\d+", l)]
            # Rough image detection via page.images (bbox)
            for im in page.images:
                # Crop image
                bbox = (im["x0"], im["top"], im["x1"], im["bottom"])
                crop = page.crop(bbox).to_image(resolution=200)
                img_bytes = io.BytesIO()
                crop.save(img_bytes, format="PNG")
                img_bytes.seek(0)
                # Attach nearest caption line (fallback empty)
                cap = caption_candidates[0] if caption_candidates else ""
                figures.append({
                    "page": pi, "bbox": bbox, "caption": cap, "image_bytes": img_bytes.getvalue()
                })
    return figures


In [8]:
# 2) Build text chunks with overlap and link figure callouts
def blocks_to_paragraphs(pages):
    paras = []
    pid = 0
    for p in pages:
        # Join block text, keep page index
        page_text = "\n".join([b[4] for b in p["blocks"] if isinstance(b[4], str) and b[4].strip()])
        # Split by double newline as a crude paragraph boundary
        for para in re.split(r"\n{2,}", page_text):
            para = para.strip()
            if len(para) > 0:
                paras.append({"paragraph_id": pid, "page": p["index"], "text": para})
                pid += 1
    return paras

def make_chunks(paras, min_tokens=180, max_tokens=600):
    # Token proxy: words count
    chunks = []
    buf = []
    wcount = 0
    for para in paras:
        words = para["text"].split()
        if wcount + len(words) > max_tokens and buf:
            chunks.append({
                "text": " ".join([b["text"] for b in buf]),
                "pages": list({b["page"] for b in buf}),
                "paragraph_ids": [b["paragraph_id"] for b in buf]
            })
            # start new buffer with overlap
            overlap = buf[-1:]
            buf = overlap + [para]
            wcount = sum(len(b["text"].split()) for b in buf)
        else:
            buf.append(para)
            wcount += len(words)
    if buf:
        chunks.append({
            "text": " ".join([b["text"] for b in buf]),
            "pages": list({b["page"] for b in buf}),
            "paragraph_ids": [b["paragraph_id"] for b in buf]
        })
    return chunks


In [11]:
!apt-get update
!apt-get install -y openjdk-21-jdk
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/java-21-openjdk-amd64/bin/java 1
!update-alternatives --config java
!java -version


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [13]:
!pip install rank_bm25


In [33]:
# 3) Embeddings + FAISS index (dense) and BM25 (sparse)
import faiss, numpy as np
from rank_bm25 import BM25Okapi

def build_dense_index(chunks, model=TEXT_EMB):
    texts = [c["text"] for c in chunks]
    vecs = model.encode(texts, normalize_embeddings=True)
    index = faiss.IndexFlatIP(vecs.shape[1])
    index.add(np.array(vecs).astype(np.float32))
    return index, vecs, texts

def build_caption_index(figures, model=CAPTION_EMB):
    caps = [f["caption"] or "" for f in figures]
    vecs = model.encode(caps, normalize_embeddings=True)
    index = faiss.IndexFlatIP(vecs.shape[1])
    index.add(np.array(vecs).astype(np.float32))
    return index, vecs, caps

def build_bm25(chunks):
    tokenized = [c["text"].split() for c in chunks]
    bm25 = BM25Okapi(tokenized)
    return bm25, tokenized

# Correct usage (unpack the tuples properly)
text_index, text_vecs, text_texts = build_dense_index(chunks)
cap_index, cap_vecs, cap_texts   = build_caption_index(figs)
bm25, tokenized                  = build_bm25(chunks)


In [27]:
# Build indices
text_index, text_vecs, text_texts = build_dense_index(chunks)
cap_index, cap_vecs, cap_texts = build_caption_index(figs)
bm25, tokenized = build_bm25(chunks)

# Example query
query = "cloud computing adoption trends"
q_vec = TEXT_EMB.encode([query], normalize_embeddings=True).astype(np.float32)

# Dense retrieval
D, I = text_index.search(q_vec, k=5)
dense_hits = [text_texts[i] for i in I[0]]

# BM25 retrieval
bm25_scores = bm25.get_scores(query.split())
top_bm25 = np.argsort(bm25_scores)[::-1][:5]
bm25_hits = [chunks[i]["text"] for i in top_bm25]


In [28]:
# 4) Retrieval: text + figure captions + simple fusion
def retrieve(query, chunks, figures, text_index, cap_index, topk=6, figk=4):
    q_text_vec = TEXT_EMB.encode([query], normalize_embeddings=True).astype(np.float32)
    D, I = text_index.search(q_text_vec, topk)
    text_hits = [{"idx": int(i), "score": float(d), "chunk": chunks[int(i)]} for d, i in zip(D[0], I[0])]

    q_cap_vec = CAPTION_EMB.encode([query], normalize_embeddings=True).astype(np.float32)
    D2, I2 = cap_index.search(q_cap_vec, figk)
    fig_hits = [{"idx": int(i), "score": float(d), "figure": figures[int(i)]} for d, i in zip(D2[0], I2[0])]

    return text_hits, fig_hits


In [34]:
# 5) Visual QA (optional): extract key facts from figures
# Using Qwen-VL or LLaVA for small QA over images
from transformers import AutoProcessor, AutoModelForVision2Seq, AutoTokenizer, AutoModelForCausalLM

def load_vlm(name="Qwen/Qwen2-VL-2B-Instruct"):
    processor = AutoProcessor.from_pretrained(name)
    model = AutoModelForVision2Seq.from_pretrained(name, device_map="auto", torch_dtype="auto")
    return processor, model

def vqa_on_figures(fig_hits, question, processor, model):
    facts = []
    for fh in fig_hits:
        img = Image.open(io.BytesIO(fh["figure"]["image_bytes"])).convert("RGB")
        prompt = f"Question: {question}\nAnswer concisely with key values, trends, axes and units."
        inputs = processor(text=prompt, images=img, return_tensors="pt").to(model.device)
        out = model.generate(**inputs, max_new_tokens=128)
        ans = processor.batch_decode(out, skip_special_tokens=True)[0]
        facts.append({"page": fh["figure"]["page"], "caption": fh["figure"]["caption"], "answer": ans})
    return facts


In [35]:
# 6) Generation: summary without chain-of-thought, with citations and figures
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_llm(name="mistralai/Mistral-7B-Instruct-v0.3"):
    tok = AutoTokenizer.from_pretrained(name, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        name, device_map="auto", torch_dtype="auto"
    )
    return tok, model

def build_prompt(query, text_hits, fig_hits, vqa_facts):
    # Assemble grounded context with locations
    text_ctx = []
    for h in text_hits:
        pages = ", ".join([str(p) for p in h["chunk"]["pages"]])
        snippet = h["chunk"]["text"][:1200]
        text_ctx.append(f"[TEXT p.{pages}] {snippet}")

    fig_ctx = []
    for f in fig_hits:
        fig_ctx.append(f"[FIGURE p.{f['figure']['page']}] {f['figure']['caption']}")

    vqa_ctx = []
    for v in vqa_facts:
        vqa_ctx.append(f"[FIGURE-FACT p.{v['page']}] {v['answer']}")

    system = (
        "You are a scientific assistant. Provide a concise summary answering the question. "
        "Do not reveal your reasoning steps. Only use provided context. "
        "Cite using page markers like [p.X] after each claim."
    )
    user = (
        f"Question: {query}\n\nContext:\n"
        + "\n".join(text_ctx[:6]) + "\n"
        + "\n".join(fig_ctx[:4]) + "\n"
        + ("\n".join(vqa_ctx) if vqa_ctx else "")
        + "\n\nInstructions:\n"
        "- Summarize without chain-of-thought.\n"
        "- Attribute claims with [p.PAGE].\n"
        "- If a figure supports a claim, cite it with [p.PAGE, figure]."
    )
    return system, user

def generate_answer(tok, model, system, user, max_new_tokens=400):
    prompt = f"<s>[SYSTEM]\n{system}\n[/SYSTEM]\n[USER]\n{user}\n[/USER]"
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    return tok.decode(out[0], skip_special_tokens=True)


In [37]:
from google.colab import files
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]  # use the actual uploaded filename
doc, pages = parse_pdf(pdf_path)
paras = blocks_to_paragraphs(pages)
chunks = make_chunks(paras)
figs = extract_figures_with_captions(pdf_path)

text_index, text_vecs, text_texts = build_dense_index(chunks)
cap_index, cap_vecs, cap_texts   = build_caption_index(figs)


query = "What are the main findings and their quantitative results?"
text_hits, fig_hits = retrieve(query, chunks, figs, text_index, cap_index)

# Optional VQA (skip if low GPU)
# vlp, vlm = load_vlm()
# vqa_facts = vqa_on_figures(fig_hits, query, vlp, vlm)
vqa_facts = []

tok, llm = load_llm()  # choose a small model if limited GPU
system, user = build_prompt(query, text_hits, fig_hits, vqa_facts)
answer = generate_answer(tok, llm, system, user)
print(answer)


Saving CLOUD_COMPUTING_2017_Athens_Greece_CLOUD.pdf to CLOUD_COMPUTING_2017_Athens_Greece_CLOUD (2).pdf


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.08 GiB. GPU 0 has a total capacity of 14.74 GiB of which 448.12 MiB is free. Process 4057 has 14.30 GiB memory in use. Of the allocated memory 14.11 GiB is allocated by PyTorch, and 73.49 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)